In [1]:
import pandas as pd
# 과학적 표기 안하도록
pd.options.display.float_format = '{:.2f}'.format

import numpy as np
import datetime

---
### request+bs로 크롤링


In [2]:
import requests
from bs4 import BeautifulSoup as bs

In [3]:
# code로 접근 가능한 url
code_url = "https://finance.naver.com/item/main.nhn?code="
# 종목 분석 url
coinfo_url = "https://finance.naver.com/item/coinfo.nhn?code="
# 종목 분석 url (!naver)
coinfo_url_2 = "https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cmp_cd=%s&cn="
# 시가총액 목록 url
size_market_url_0 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="
size_market_url_1 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page="

---

### 시가총액 목록 사용하기

In [4]:
# 시가총액 300억 이하 목록 구하기
codeL = []

# 코스피 300억 이하 (후보)
# for i in range(19, 33):
for i in range(17, 33):
    soup = bs(requests.get(size_market_url_0+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL

# 코스닥 300억 이하 (후보)
# for i in range(22, 30):
for i in range(19, 30):
    soup = bs(requests.get(size_market_url_1+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL
    

In [5]:
# 오늘의 시가총액 code 목록 저장 
todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
f = open("./"+todayDate+"_시가총액_하위권.txt", 'w')
f.write("\n".join(codeL))
f.close()

# 오늘의 시가총액 code 목록 불러오기
useDate = todayDate
# useDate = "2020_Feb_18"
f = open("./"+useDate+"_시가총액_하위권.txt", "r")
codeL = [x.replace("\n", "") for x in f.readlines()]
f.close()

---

<b>base code</b>


In [20]:
def getSoupFromCode(code):
    code_url = "https://finance.naver.com/item/main.nhn?code="
    soup = bs(requests.get(code_url + code).text, 'html.parser')
    return soup

def cleanText(text):
    return text.replace("\n", "").replace("\t","").replace("조","").replace(",","")

def getName(code):
    soup = getSoupFromCode(code)
    name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
    return name

def getSoupFromReport(code):
    report_url = "https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd="
    soup = bs(requests.get(report_url + code).text, 'html.parser')
    return soup

def getMeanPer(code):
    soup = getSoupFromReport(code)
    try:
        temp = [x.text.split()[1] for x in soup.select_one(".td0301").select(".line-left") if x.text.split()[0] == "업종PER"]
    except:
        return 0
    per_str = [x.text.split()[1] for x in soup.select_one(".td0301").select(".line-left") if x.text.split()[0] == "업종PER"][0]
    if(per_str == "N/A"):
        return 0
    return per_str


---

## 사용할 조건들

- PBR 1 미만
- PER 7 미만
- 시총 300 미만
- PCR 8 미만
- POR 8 미만


---

#### etf 인가

In [7]:
def isETF(code):
    soup = getSoupFromCode(code)
    if(len(soup.select(".e_summary")) == 0):
        return False
    return True

#### etn 인가

In [8]:
def isETN(code):
    name = getName(code)
    if("ETN" in name):
        return True
    return False

---

#### 우선주인가


In [14]:
def isFirst(code):
    name = getName(code)
    if(name[-1] == "우"):
        return True
    if(name[-2:] == "우A"):
        return True
    if(name[-2:] == "우B"):
        return True
    if(name[-2:] == "우C"):
        return True
    return False

#### 시총 300억 미만인가

In [9]:
def under300(code):
    soup = getSoupFromCode(code)
    
    if(int(cleanText(soup.select_one("#_market_sum").text)) < 300):
        return True
    return False    

#### per 7 미만인가

In [24]:
def perUnder7(code):
    soup = getSoupFromReport(code)
    try:
        temp = [x.text.split()[1] for x in soup.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PER"]
    except:
        return -1
    per_str = [x.text.split()[1] for x in soup.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PER"][0]
    if(per_str == "N/A"):
        return -1
    per_str = cleanText(per_str)
    if(float(per_str) < 7 and float(per_str) > 0):
        return per_str
    return -1
    

#### pbr 1 미만인가

In [25]:
def pbrUnder1(code):
    soup = getSoupFromReport(code)
    try:
        temp = [x.text.split()[1] for x in soup.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PBR"]
    except:
        return -1
    pbr_str = [x.text.split()[1] for x in soup.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PBR"][0]
    pbr_str = cleanText(pbr_str)
    if(float(pbr_str) < 1 and float(pbr_str) > 0):
        return pbr_str
    return -1


---

## 괜찮은 종목 거르기

In [26]:
# codeL.reverse()

In [28]:
good = []

for code in codeL:
    if(isETF(code)):
        continue
    if(isETN(code)):
        continue
    if(isFirst(code)):
        continue
    per = perUnder7(code)
    if(per == -1):
        continue
    pbr = pbrUnder1(code)
    if(pbr == -1):
        continue
    name = getName(code)
    good.append(name + " " + str(per) + " " + str(pbr))
    print(name, per, pbr)

파티게임즈 0.94 0.58
영신금속 2.87 0.39
PN풍년 3.21 0.43
동일철강 4.51 0.28
패션플랫폼 5.59 0.57
에스앤씨엔진그룹 2.10 0.05
제일테크노스 5.98 0.30
피앤텔 0.52 0.38
성우테크론 6.22 0.54
신원종합개발 1.98 0.29
휴맥스홀딩스 4.94 0.16
원일특강 3.10 0.26
한국컴퓨터 4.23 0.32
코리아에셋투자증권 4.42 0.51
아즈텍WB 6.83 0.34
KBI메탈 5.19 0.37
제이스테판 2.57 0.07
스페코 3.84 0.93
DSC인베스트먼트 5.51 0.96
흥국 5.06 0.52
유비벨록스 5.98 0.44
바이오로그디바이스 6.18 0.69
에코바이오 5.39 0.62
오리콤 6.50 0.46
파워넷 6.49 0.65
에스에이티 4.36 0.48
한일네트웍스 5.89 0.62
골든센츄리 1.09 0.14
케이디켐 5.68 0.53
슈프리마에이치큐 5.40 0.27
원익 0.94 0.39
삼진 3.81 0.60
에스폴리텍 4.36 0.80
현대통신 4.51 0.58
메디앙스 1.24 0.59
선익시스템 6.89 0.52
케이피에프 6.30 0.29
SG 4.73 0.68
푸드웰 1.58 0.62
흥국화재2우B 4.07 0.22
넥센타이어1우B 4.27 0.32
에이리츠 1.51 0.63
하이골드3호 2.40 0.54
이스타코 3.13 0.46
두산2우B 1.95 0.40
하이골드12호 3.79 0.78
하이골드8호 4.99 0.93
한솔로지스틱스 5.37 0.67
SJM 4.99 0.21
동원수산 6.14 0.73
신풍제지 1.98 0.44
진양산업 5.88 0.98
티에이치엔 2.48 0.68
한국프랜지 3.66 0.19
동양피스톤 4.34 0.39
NI스틸 6.73 0.39
DSR제강 6.90 0.33
까뮤이앤씨 3.55 0.66
CS홀딩스 5.19 0.22
미래아이앤지 3.69 0.74
경동인베스트 5.83 0.12
DSR 6.22 0.35


In [25]:
len(good)

39

In [ ]:
good

---

## 나의 네이버증권 셋팅하기

In [13]:
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys

import time

In [14]:
# 미리 로그인
driver = webdriver.Chrome('./chromedriver')
itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL"
driver.get(itemListUrl)

In [16]:
for group in range(6, 9):

    # 링크 접속 및 로그인
    itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=%s&type=OVERALL" % str(group)
    print(itemListUrl )
    
    # 기존 관심 종목 삭제
    time.sleep(0.5)
    driver.get(itemListUrl)
    time.sleep(0.5)
    driver.find_element_by_css_selector("#all_select").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector("#wrap > div.section_mys > div.group_mystb.NE\=a\:lst > div:nth-child(2) > button.btn_del.NPI\=a\:del").click()
    time.sleep(0.5)
    Alert(driver).accept()
    
    # 적합 종목 리스트 찾기
    if(group == 6):
        useL = score25_code.copy()
    elif(group == 7):
        useL = score23_code.copy()
    elif(group == 8):
        useL = score20_code.copy()
            
    # 관심종목 목록 등록
    for nowCode in useL:
        time.sleep(0.5)
        driver.find_element_by_css_selector("#mystock_input").clear()
        time.sleep(0.5)
        driver.find_element_by_css_selector("#mystock_input").send_keys(nowCode)
        driver.find_element_by_css_selector("#mystock_input").send_keys(Keys.RETURN)


https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL
https://finance.naver.com/mystock/itemList.nhn?groupId=7&type=OVERALL
https://finance.naver.com/mystock/itemList.nhn?groupId=8&type=OVERALL
